In [1]:
import ollama
import requests
import json

In [19]:
model_name = "llama3.1"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a LLMChain using with a Ollama model.
- Define a prompt template for summarization.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with a sample text and print the summary.


In [20]:
from langchain.chains import LLMChain
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate

# Load the Ollama model
llm = OllamaLLM(model=model_name)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in one sentence:\n{text}"
)

# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""
summary = chain.run(text)
print(summary)

Over the past decade, significant advances in deep learning have led to AI's widespread adoption and potential major impact on businesses, societies, and individuals due to technological advancements, notable applications, and transformative sectoral effects.


In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Load the Ollama model
llm = OllamaLLM(model=model_name)

# Define the prompt template
template = """Summarize the following text in one sentence: {text}

"""

prompt = ChatPromptTemplate.from_template(template)

# Create the LLMChain
chain = prompt | llm

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""
summary = chain.invoke({"text": text})
print(summary)

Over the past decade, deep learning has advanced significantly, enabling AI models to process and generate unstructured data, and is poised to have a major impact on businesses, societies, and individuals.


In [25]:
# Stream the chain output
for chunk in chain.stream({"text": text}):
    print(chunk, end="", flush=True)#

Here is a summary of the text in one sentence:

Over the last decade, deep learning has significantly advanced AI models, particularly large language models, which are poised to have a major impact on businesses, societies, and individuals.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Create an LLMChain using an Ollama model.
- Define a prompt template that includes a placeholder for the math operation.
- Define a tool a simple math tool.
- Run the chain with a prompt that requires the math tool, passing the placeholder to the tool wrapper.
- Print the result of the calculation.

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools (e.g., a math tool and a customer service tool) to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.

#### Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.